In [4]:
from tensorflow.keras.models import load_model
import numpy as np
import os
import sys
import pathlib
from pathlib import Path

In [12]:
current_dir = os.getcwd()
parent_dir = os.path.dirname(current_dir)
sys.path.append(parent_dir)
current_file_path = pathlib.Path(os.getcwd())
parent_dir = current_file_path.parent
directory_path = parent_dir / 'dataset'
import swallowing_recognition

In [9]:
from swallowing_recognition import dataset

In [34]:
loaded_model = load_model('20240205_binary_ep50_bs32.keras')

# フォルダのファイルを全て読み込み，予測する

In [20]:
name = "tsuji"

In [17]:
prediction_swallowing_folder = directory_path / name / 'swallowing'
prediction_voice_folder = directory_path / name / 'voice'
prediction_cough_folder = directory_path / name / 'cough'
prediction_hitting_folder = directory_path / name / 'hitting'    
prediction_movement_folder = directory_path / name / 'movement'    

In [18]:
prediction_data = dataset.DataSet(50, 224, 224, 3, 2)
prediction_data.folder_to_dataset(prediction_swallowing_folder, np.array(0), 0)
prediction_data.folder_to_dataset(prediction_voice_folder, np.array(1), 10)
prediction_data.folder_to_dataset(prediction_cough_folder, np.array(1), 20)
prediction_data.folder_to_dataset(prediction_hitting_folder, np.array(1), 30)
prediction_data.folder_to_dataset(prediction_movement_folder, np.array(1), 40)

In [40]:
predictions = loaded_model.predict(prediction_data.data)
predicted_classes = (predictions > 0.5).astype(int)
predicted_classes = np.squeeze(predicted_classes)
print("Predicted classes:", predicted_classes)

2/2 [==============================] - 0s 26ms/step
Predicted classes: [1 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 0 1 1 0 1 1 1 1 1]


In [46]:
from sklearn.metrics import precision_score, recall_score, f1_score
precision = precision_score(prediction_data.labels, predicted_classes)
recall = recall_score(prediction_data.labels, predicted_classes)
f1 = f1_score(prediction_data.labels, predicted_classes)

print("適合率（Precision）: {:.2f}".format(precision))
print("再現率（Recall）: {:.2f}".format(recall))
print("F値: {:.2f}".format(f1))

適合率（Precision）: 0.97
再現率（Recall）: 0.93
F1値: 0.95
